In [4]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path

model = SentenceTransformer("all-MiniLM-L6-v2")


/opt/anaconda3/envs/skn19_final/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
THRESHOLD = 0.9

# 누적 저장소
stored_indexes = []
stored_texts = []
stored_embeddings = []

In [37]:
def normalize_index(s: str) -> str:
    return s.replace(".", " ").lower()


def process_json_file(json_path: Path):
    with open(json_path, "r", encoding="utf-8") as f:
        items = json.load(f)

    # 🔹 현재 파일 전용 저장소
    local_indexes = []
    local_texts = []
    local_embeddings = []

    for order, item in enumerate(items):
        raw_index = f"""
            index : {item["index"]}
            text : {item["text"]}
        """
        text = item["index"]

        index_for_embedding = normalize_index(raw_index)

        # 1️⃣ index 임베딩
        embedding = model.encode(
            index_for_embedding,
            convert_to_numpy=True,
            normalize_embeddings=True
        )

        # 2️⃣ 이전 파일들과만 비교
        if stored_embeddings:
            sims = cosine_similarity(
                [embedding],
                np.vstack(stored_embeddings)
            )[0]

            for i, score in enumerate(sims):
                if score >= THRESHOLD:
                    print("🔥 유사 항목 발견")
                    print(f"- 현재 파일 : {json_path.name}")
                    print(f"- 현재 index : {raw_index}")
                    print(f"- 기존 index : {stored_indexes[i]}")
                    print(f"- 유사도 : {score:.3f}")
                    print("- 현재 text:")
                    print(text.strip())
                    print("- 기존 text:")
                    print(stored_texts[i].strip())
                    print("-" * 80)

        # 3️⃣ 현재 파일 항목은 local에만 저장
        local_indexes.append(raw_index)
        local_texts.append(text)
        local_embeddings.append(embedding)

    # ✅ 파일 처리 종료 후 전역 저장소에 병합
    stored_indexes.extend(local_indexes)
    stored_texts.extend(local_texts)
    stored_embeddings.extend(local_embeddings)


In [ ]:
json_dir = Path("docs_data/indexing_data")

for json_file in sorted(json_dir.glob("*.jsonl")):
    process_json_file(json_file)
    print("저장 완료")

저장 완료
🔥 유사 항목 발견
- 현재 파일 : indexing_file_005.json
- 현재 index : 
            index : 기안서.핵심솔루션.투명한지식계보관리.기여도추적
            text : * **기여도 추적:** 병합된 문서에서도 최초 작성자를 추적하여 지식 기여에 대한 공정한 보상 체계 마련.

---


        
- 기존 index : 
            index : 문서등록로직.💾문서등록로직
            text : ## 💾 문서 등록 로직


        
- 유사도 : 0.873
- 현재 text:
기안서.핵심솔루션.투명한지식계보관리.기여도추적
- 기존 text:
문서등록로직.💾문서등록로직
--------------------------------------------------------------------------------
🔥 유사 항목 발견
- 현재 파일 : indexing_file_005.json
- 현재 index : 
            index : 기안서.핵심솔루션.투명한지식계보관리.기여도추적
            text : * **기여도 추적:** 병합된 문서에서도 최초 작성자를 추적하여 지식 기여에 대한 공정한 보상 체계 마련.

---


        
- 기존 index : 
            index : 문서생성로직.입력및데이터조회.처리.문서생성
            text : ## 📊 문서 생성 로직


        
- 유사도 : 0.885
- 현재 text:
기안서.핵심솔루션.투명한지식계보관리.기여도추적
- 기존 text:
문서생성로직.입력및데이터조회.처리.문서생성
--------------------------------------------------------------------------------
🔥 유사 항목 발견
- 현재 파일 : indexing_file_005.json
- 현재 index : 
            inde

In [ ]:
import json
from typing import List, Tuple
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


def find_similar_pairs_from_jsonl(
    jsonl_path: str,
    text_key: str = "text",
    model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
    similarity_threshold: float = 0.85,
    max_pairs: int | None = None
) -> List[dict]:
    """
    jsonl 파일에서 text를 임베딩한 뒤,
    cosine similarity가 threshold 이상인 쌍을 반환한다.

    Returns:
        [
            {
                "idx1": int,
                "idx2": int,
                "similarity": float,
                "text1": str,
                "text2": str
            },
            ...
        ]
    """

    # 1. jsonl 로드
    texts = []
    idxs = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            if text_key in obj and obj[text_key].strip():
                texts.append(obj[text_key])
                idxs.append(obj["index"])

    if len(texts) < 2:
        return []

    # 2. 임베딩 모델 로드
    model = SentenceTransformer(model_name)

    # 3. 임베딩
    embeddings = model.encode(
        texts,
        batch_size=32,
        convert_to_tensor=True,
        normalize_embeddings=True
    )

    sim_matrix = torch.matmul(embeddings, embeddings.T)

    results = []
    n = len(texts)

    for i in range(n):
        for j in range(i + 1, n):
            sim = sim_matrix[i, j].item()

            if sim >= similarity_threshold:
                if len(texts[i]) > 10 and len(texts[j]) > 10 :
                    results.append({
                            "idx1": idxs[i],
                            "idx2": idxs[j],
                            "similarity": round(sim, 4),
                            "text1": texts[i],
                            "text2": texts[j],
                            "idx1" : idxs[i],
                            "idx2" : idxs[j]
                        })

    # 유사도 높은 순으로 정렬
    results.sort(key=lambda x: x["similarity"], reverse=True)

    if max_pairs is not None:
        results = results[:max_pairs]

    return results


In [11]:
pairs = find_similar_pairs_from_jsonl(
    jsonl_path="docs_data/merged_data/index_data.jsonl",
    similarity_threshold=0.8
)

for p in pairs:
    print(p["similarity"])
    print("A:", p["text1"])
    print("A index : ", p["idx1"])
    print("B:", p["text2"])
    print("B index : ", p["idx2"])
    print("-" * 50)


0.9296
A: 

        pgsql
        코드 복사

A index :  스팀오리프로그램원리.pgsql코드복사잔여텍스트처리섹션
B: 

        pgsql
        코드 복사

B index :  LSTM변형모델.LSTM변형모델.pgsql및코드복사관련이슈정리.pgsql코드복사잔여텍스트처리
--------------------------------------------------
0.8948
A: ## Reference

{url_027}

{url_024}


        pgsql
        코드 복사

A index :  BedrockAgent로Chatbot구현.BedrockAgent의동작개요섹션.pgsql코드복사잔여텍스트처리섹션.pgsql코드복사잔여텍스트처리
B: ## Reference

{url_003}


        pgsql
        코드 복사

B index :  검색증강생성(RAG)개념및임베딩기반검색·청킹전략섹션.LLM문서전처리·청킹전략개요섹션.pgsql코드복사잔여텍스트처리섹션.pgsql코드복사잔여텍스트처리
--------------------------------------------------
0.8783
A: ## 5. 기대 효과 (Expected Benefits)

* **운영 효율:** 중복 문서 정리 및 검색 시간 단축을 통해 업무 효율성 향상.
* **보안 완결:** 사내 기밀 유출 걱정 없는 안전한 AI 활용 환경 구축.
* **지식 자산화:** 개인의 노하우가 조직의 공식 지식으로 자동 승격되는 자생적 생태계 조성.

---

A index :  file005.AJC활용시나리오.기대효과운영효율보안지식자산화
B: ## 5. 기대 효과 (Expected Benefits)

* **운영 효율:** 중복 문서 정리 및 검색 시간 단축을 통해 업무 효율성 향상.
* **보안 완결:** 사내 기밀 유출 걱정 없는 안전한 AI 활용 환경 구축.
* **지식 자산화:** 개인의 노하우가 조직의 공식 